In [29]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [32]:
%%sql
drop table if exists Sales, Titles, Newsstands;
CREATE TABLE Titles
(product_id INTEGER NOT NULL PRIMARY KEY,
 magazine_sku INTEGER NOT NULL,
 issn INTEGER NOT NULL,
 issn_year INTEGER NOT NULL);

CREATE TABLE Newsstands
 (stand_nbr INTEGER NOT NULL PRIMARY KEY,
  stand_name CHAR(20) NOT NULL);

CREATE TABLE Sales
(product_id   INTEGER NOT NULL REFERENCES Titles(product_id),
 stand_nbr    INTEGER NOT NULL REFERENCES Newsstands(stand_nbr),
 net_sold_qty INTEGER NOT NULL,
 PRIMARY KEY(product_id, stand_nbr));

INSERT INTO Titles VALUES(1, 12345, 1, 2006);
INSERT INTO Titles VALUES(2, 2667,  1, 2006);
INSERT INTO Titles VALUES(3, 48632, 1, 2006);
INSERT INTO Titles VALUES(4, 1107,  1, 2006);
INSERT INTO Titles VALUES(5, 12345, 2, 2006);
INSERT INTO Titles VALUES(6, 2667,  2, 2006);
INSERT INTO Titles VALUES(7, 48632, 2, 2006);
INSERT INTO Titles VALUES(8, 1107,  2, 2006);

INSERT INTO Newsstands VALUES(1, 'Newsstands1');
INSERT INTO Newsstands VALUES(2, 'Newsstands2');
INSERT INTO Newsstands VALUES(3, 'Newsstands3');
INSERT INTO Newsstands VALUES(4, 'Newsstands4');

INSERT INTO Sales VALUES(1, 1, 1);
INSERT INTO Sales VALUES(2, 1, 4);
INSERT INTO Sales VALUES(3, 1, 1);
INSERT INTO Sales VALUES(4, 1, 1);
INSERT INTO Sales VALUES(5, 1, 1);
INSERT INTO Sales VALUES(6, 1, 2);
INSERT INTO Sales VALUES(7, 1, 1);
INSERT INTO Sales VALUES(3, 2, 1);
INSERT INTO Sales VALUES(4, 2, 5);
INSERT INTO Sales VALUES(8, 2, 6);
INSERT INTO Sales VALUES(1, 3, 1);
INSERT INTO Sales VALUES(2, 3, 3);
INSERT INTO Sales VALUES(3, 3, 3);
INSERT INTO Sales VALUES(4, 3, 1);
INSERT INTO Sales VALUES(5, 3, 1);
INSERT INTO Sales VALUES(6, 3, 3);
INSERT INTO Sales VALUES(7, 3, 3);
INSERT INTO Sales VALUES(1, 4, 1);
INSERT INTO Sales VALUES(2, 4, 1);
INSERT INTO Sales VALUES(3, 4, 4);
INSERT INTO Sales VALUES(4, 4, 1);
INSERT INTO Sales VALUES(5, 4, 1);
INSERT INTO Sales VALUES(6, 4, 1);
INSERT INTO Sales VALUES(7, 4, 2);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [15]:
%%sql
select stand_nbr
from Sales as s
right join Titles as t
on s.product_id = t.product_id
group by stand_nbr
having (
    avg(
        case when t.magazine_sku = 2667 then s.net_sold_qty else null end
    ) > 2 and
    avg(
        case when t.magazine_sku = 48632 then s.net_sold_qty else null end
    ) > 2
) or (
    avg(
        case when t.magazine_sku = 1107 then s.net_sold_qty else null end
    ) > 5
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


stand_nbr
3
2


In [54]:
%%sql
select stand_nbr
from (
    select s.stand_nbr, t.magazine_sku, avg(s.net_sold_qty)
    from Sales as s, Titles as t
    where s.product_id = t.product_id
    group by s.stand_nbr, t.magazine_sku
) as s
group by stand_nbr
having count(case when magazine_sku in (2667, 48632) and avg > 2 then 1 else null end) = 2 or
count(case when magazine_sku = 1107 and avg > 5 then 0 else null end) = 1

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


stand_nbr
3
2


In [48]:
%%sql
-- havingをselectにもってくる方法
select stand_nbr
from (
    select stand_nbr,
        avg(
            case when t.magazine_sku = 2667 then s.net_sold_qty else null end
        ),
        avg(
            case when t.magazine_sku = 48632 then s.net_sold_qty else null end
        ),
        avg(
            case when t.magazine_sku = 1107 then s.net_sold_qty else null end
        )
    from Sales as s, Titles as t
    where s.product_id = t.product_id
    group by stand_nbr
) as tmp(stand_nbr, avg_2667, avg_48632, avg_1107)
where avg_2667 > 2 and avg_48632 > 2 or avg_1107 > 5

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


stand_nbr
3
2


In [26]:
%%sql
select stand_nbr
from (
    select s.stand_nbr, t.magazine_sku, avg(s.net_sold_qty)
    from Sales as s
    right join Titles as t
    on s.product_id = t.product_id
    where t.magazine_sku in (2667, 48632)
    group by s.stand_nbr, t.magazine_sku
    having avg(s.net_sold_qty) >2
) as tmp
group by stand_nbr
having count(*) = 2

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


stand_nbr
3


In [9]:
%%sql
select t.magazine_sku
from Titles as t
inner join Sales as s
on s.product_id = t.product_id
where t.magazine_sku in (2667, 48632)
group by t.magazine_sku
having avg(s.net_sold_qty) > 2

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


magazine_sku
2667
48632
